# Captcha Project - Dataset & MLflow Setup

이 노트북은 데이터셋을 다운로드하고, MLflow 서버를 실행하며, ngrok을 통해 외부 주소를 생성합니다.

In [ ]:
import os
import sys
import subprocess
import glob
import shutil
from pathlib import Path
import time
import socket

import kagglehub
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

from skimage.io import imread
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from skimage.feature import hog
from skimage.transform import resize
from skimage.morphology import opening, footprint_rectangle

import mlflow
from pyngrok import ngrok
from tqdm.notebook import tqdm

## 1. 데이터셋 다운로드 & 경로 설정

In [ ]:
# kagglehub를 이용해서 최신 버전의 데이터셋을 다운로드
dataset_path = kagglehub.dataset_download("fournierp/captcha-version-2-images")
print("Path to dataset files:", dataset_path)

# 이미지가 들어있는 디렉터리
images_dir = Path(dataset_path) / "samples"
image_files = list(images_dir.glob("*.png")) + list(images_dir.glob("*.jpg"))
print(f"찾은 이미지 개수: {len(image_files)}개")

## 2. MLflow & ngrok 설정 (서버 실행)

In [ ]:
# 기존 프로세스 완전 청소 (ERR_NGROK_334 방지)
try:
    if os.name == 'posix':
        !pkill -f ngrok
        !pkill -f mlflow
    ngrok.kill()
except Exception:
    pass

# MLflow 설정
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("Captcha_Project_V1")

# MLflow 서버 실행 (배경실행 - macOS AirPlay 충돌 방지를 위해 5001 포트 사용)
# --allowed-hosts "*" 옵션을 추가하여 ngrok을 통한 접속이 가능하도록 합니다.
get_ipython().system_raw(f"{sys.executable} -m mlflow ui --port 5001 --host 0.0.0.0 --allowed-hosts '*' &")

# 서버 부팅 대기 (Health Check)
print("MLflow 서버 부팅 중...")
for _ in range(15):
    time.sleep(1)
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    result = sock.connect_ex(('127.0.0.1', 5001))
    if result == 0:
        print("✅ MLflow 서버가 정상적으로 응답합니다!")
        break
    sock.close()
else:
    print("❌ 서버 부팅 시간이 너무 길어집니다. 다시 실행해 보세요.")

## 3. ngrok 터널 활성화

In [ ]:
NGROK_AUTH_TOKEN = "34lM1T5TWYWgnNcWPYgk7OLcLQb_4V3iPfdh1wRmCW3ykat2p"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

try:
    # 5001번 포트를 외부로 개방
    public_url = ngrok.connect(5001).public_url
    print(f"\n🚀 아래 주소를 클릭하세요:\n{public_url}")
    print("\n⚠️  접속 후 'Visit Site' 버튼이 나오면 클릭하세요.")
except Exception as e:
    print(f"❌ ngrok 연결 중 오류 발생: {e}")

In [ ]:
# ============================
# 2. 유틸 함수들 정의
# ============================

def extract_label_from_filename(file_path: Path) -> str:
    """
    파일 이름에서 레이블(정답 텍스트)을 추출하는 함수.
    - 실제 데이터셋의 파일명 규칙에 맞게 수정해야 함.\n",
    - 예시: '2b7wD.png' -> '2b7wD'\n",
    """
    # 확장자 제거 후 파일명 전체를 레이블로 사용 (필요 시 슬라이싱/분리)
    label = file_path.stem
    return label


def preprocess_image(img):
    if img.ndim == 3 and img.shape[2] == 4:
        from skimage.color import rgba2rgb
        img = rgba2rgb(img)
    
    img_gray = rgb2gray(img)
    img_resized = resize(img_gray, (50, 200))
    
    thresh = threshold_otsu(img_resized)
    # 글자가 배경보다 어둡다고 가정 (검정 글씨: False, 흰 배경: True)
    img_bin = img_resized > thresh
    # 만약 이미지 가장자리가 검은색이면 반전 (배경을 항상 True(흰색)로 유지)
    if np.mean(img_bin[0, :]) < 0.5:
        img_bin = ~img_bin
        
    return img_bin

def preprocess_image_v2(img):
    # 기본 전처리 (RGB2Gray -> Resize -> Otsu)
    if img.ndim == 3 and img.shape[2] == 4: img = img[..., :3]
    img_gray = rgb2gray(img)
    img_resized = resize(img_gray, (50, 200))
    thresh = threshold_otsu(img_resized)
    img_bin = img_resized > thresh
    # 노이즈(선) 제거: 아주 작은 점이나 얇은 선을 지웁니다.
    # 1(배경), 0(글자)이므로 글자를 지우기 위해 반전 후 처리
    if np.mean(img_bin[0, :]) < 0.5: img_bin = ~img_bin
    
    # 노이즈(선) 제거
    img_cleaned = opening(1 - img_bin, footprint_rectangle((2, 2))) 
    return 1 - img_cleaned


def extract_hog_features(img_binary: np.ndarray) -> np.ndarray:
    """
    HOG 특징 추출
    """
    features = hog(
        img_binary,
        orientations=9, # 그래디언트(기울기)의 방향을 몇 개의 구간(Bin)으로 나눌 것인지를 결정
        pixels_per_cell=(4, 4), # 하나의 셀(Cell)을 구성하는 픽셀의 크기
        cells_per_block=(2, 2), # 하나의 블록(Block)에 포함될 셀의 개수
        block_norm="L2-Hys", # 블록 단위 정규화 방식을 설정 (L2-Hys: L2-norm, clipping(Hysteresis)의 약자)
        transform_sqrt=True # 입력 이미지에 대해 감마 보정에 대한 유무

    )
    return features

def segment_characters(img_bin, num_chars=5):
    """
    이미지를 단순하게 등분하여 글자를 분리합니다.
    """
    h, w = img_bin.shape
    char_width = w // num_chars
    
    characters = []
    for i in range(num_chars):
        start = i * char_width
        end = (i + 1) * char_width
        
        char_img = img_bin[:, start:end]
        char_img_resized = resize(char_img, (32, 32))
        characters.append(char_img_resized)
        
    return characters

In [ ]:
# ============================
# 3. 데이터 로딩 & 특징 추출
# ============================

IMAGE_EXTS = ["*.png", "*.jpg", "*.jpeg"]

image_files = []
for ext in IMAGE_EXTS:
    image_files.extend(list(images_dir.glob(ext)))

print(f"총 이미지 개수: {len(image_files)}")

X_char_features = []
y_char_labels = []

for path in tqdm(image_files, desc="특징 추출 중"):
    try:
        img = imread(str(path))
        img_bin = preprocess_image_v2(img)
        char_images = segment_characters(img_bin, num_chars=5)
        label_text = path.stem 
        
        if len(char_images) == len(label_text):
            for char_img, char_label in zip(char_images, label_text):
                feat = extract_hog_features(char_img)
                X_char_features.append(feat)
                y_char_labels.append(char_label)
                
    except Exception as e:
        continue

X_char_features = np.array(X_char_features)
y_char_labels = np.array(y_char_labels)

print(f"총 추출된 글자 데이터 수: {len(X_char_features)}")

In [ ]:
# ============================
# 4. 레이블 인코딩 & 학습/검증 분할
# ============================

X_char_features = np.array(X_char_features)
y_char_labels = np.array(y_char_labels)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_char_labels)

n_classes = len(label_encoder.classes_)
print(f"인코딩된 클래스 개수: {n_classes}")

X_train, X_test, y_train, y_test = train_test_split(
    X_char_features,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)

# 특징 벡터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("-" * 30)
print(f"학습용 글자 샘플 수: {X_train.shape[0]}")
print(f"검증용 글자 샘플 수: {X_test.shape[0]}")

In [ ]:
# ============================
# 5. 모델 학습 및 평가 (with MLflow)
# ============================

with mlflow.start_run(run_name="Improved_HOG_SVM_Run"):
    mlflow.log_param("max_iter", 10000)
    mlflow.log_param("model_type", "SVC_RBF")
    mlflow.log_param("C", 1.0)
    mlflow.log_param("hog_pixels_per_cell", "(4, 4)")
    
    svm_model = SVC(kernel='rbf', C=10.0, gamma='scale', random_state=42)
    svm_model.fit(X_train_scaled, y_train)

    y_pred = svm_model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)

    mlflow.log_metric("accuracy", accuracy)

    print(f"개선된 모델 정확도: {accuracy:.2%}")
    print("\n[상세 분류 리포트]")
    report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)
    print(report)
    
    mlflow.sklearn.log_model(svm_model, "captcha_model")
    
    joblib.dump(svm_model, "../models/captcha_svm_model.pkl")
    joblib.dump(label_encoder, "../models/label_encoder.pkl")
    joblib.dump(scaler, "../models/scaler.pkl")
    print("모델, 레이블 저장 완료")